In [ ]:
%pip install --upgrade --quiet boto3==1.34.162 matplotlib scikit-learn tqdm langchain langchain-community langchain-experimental langchain_aws nest_asyncio json-repair awscli numpy pandas gremlinpython config streamlit
%pip install --upgrade --quiet PyPDF2

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install gremlinpython
%pip install nest_asyncio


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain.prompts import PromptTemplate
from langchain_community.graphs.neptune_graph import NeptuneGraph
import config as cf
import os
import base64
from gremlin_python.driver import client, serializer
# Additional libraries for data processing
import pandas as pd

neptune_endpoint = 'neptunedbcluster-vugbwk7upvfa.cluster-cdpzhcjwte6u.us-east-1.neptune.amazonaws.com'
neptune_port = '8182'

# Create a Neptune client
neptune_client = client.Client(
    f'wss://{neptune_endpoint}:{neptune_port}/gremlin',
    'g',
    message_serializer=serializer.GraphSONSerializersV2d0()
)


In [ ]:
prompting = """# Knowledge Graph Instructions for Claude 3.5
## 1. Overview
You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph. You are a data scientist working for a company that is building a graph database. Your task is to extract information from data and convert it into three words with meaning Source_Node, Relationship, Target_Node. The first and third words are to be used as nodes and the second word is to be used as relationship. So, maintain consistency in spelling and capital letter such that node with same name is formed only once later.
- **Nodes** represent entities and concepts. They're akin to Wikipedia nodes.
- The aim is to achieve simplicity and clarity in the knowledge graph, making it accessible for a vast audience.
## 2. Labeling Nodes
- **Consistency**: Ensure you use basic or elementary types for node labels.
  - For example, when you identify an entity representing a person, always label it as **"person"**. Avoid using more specific terms like "mathematician" or "scientist".
- **Node IDs**: Never utilize integers as node IDs. Node IDs should be names or human-readable identifiers found in the text.
## 3. Handling Numerical Data and Dates
- Numerical data, like age or other related information, should be incorporated as attributes or properties of the respective nodes.
- **No Separate Nodes for Dates/Numbers**: Do not create separate nodes for dates or numerical values. Always attach them as attributes or properties of nodes.
- **Property Format**: Properties must be in a key-value format.
- **Quotation Marks**: Never use escaped single or double quotes within property values.
- **Naming Convention**: Use camelCase for property keys, e.g., `birthDate`.
## 4. Coreference Resolution
- **Maintain Entity Consistency**: When extracting entities, it's vital to ensure consistency.
If an entity, such as "John Doe", is mentioned multiple times in the text but is referred to by different names or pronouns (e.g., "Joe", "he"), 
always use the most complete identifier for that entity throughout the knowledge graph. In this example, use "John Doe" as the entity ID.  
Remember, the knowledge graph should be coherent and easily understandable, so maintaining consistency in entity references is crucial. 
## 5. Strict Compliance
Adhere to the rules strictly. Non-compliance will result in termination.

<rules>
Only return this list and nothing else. 
No extra words or sentences.
Do not add "Here is the list of key points from the document:"
</rules>

Example:
Data: Alice lawyer and is 25 years old and Bob is her roommate since 2001. Bob works as a journalist. Alice owns a the webpage www.alice.com and Bob owns the webpage www.bob.com.

List created is given below. 

\n\nAssistant:
['Alice~~is~~Lawyer', 'Alice~~age~~25years', 'Alice~~roommate~~Bob', 'Bob~~is~~Journalist', 'Alice~~owns~~www.alice.com', 'Bob~~owns~~www.bob.com']


"""

prompting_image = """# Knowledge Graph Instructions for Claude 3.5
## 1. Overview
You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph. You are a data scientist working for a company that is building a graph database. Your task is to extract information from data and convert it into three words with meaning Source_Node, Relationship, Target_Node. The first and third words are to be used as nodes and the second word is to be used as relationship. So, maintain consistency in spelling and capital letter such that node with same name is formed only once later.
- **Nodes** represent entities and concepts. They're akin to Wikipedia nodes.
- The aim is to achieve simplicity and clarity in the knowledge graph, making it accessible for a vast audience.
## 2. Labeling Nodes
- **Consistency**: Ensure you use basic or elementary types for node labels.
  - For example, when you identify an entity representing a person, always label it as **"person"**. Avoid using more specific terms like "mathematician" or "scientist".
- **Node IDs**: Never utilize integers as node IDs. Node IDs should be names or human-readable identifiers found in the text.
## 3. Handling Numerical Data and Dates
- Numerical data, like age or other related information, should be incorporated as attributes or properties of the respective nodes.
- **No Separate Nodes for Dates/Numbers**: Do not create separate nodes for dates or numerical values. Always attach them as attributes or properties of nodes.
- **Property Format**: Properties must be in a key-value format.
- **Quotation Marks**: Never use escaped single or double quotes within property values.
- **Naming Convention**: Use camelCase for property keys, e.g., `birthDate`.
## 4. Coreference Resolution
- **Maintain Entity Consistency**: When extracting entities, it's vital to ensure consistency.
If an entity, such as "John Doe", is mentioned multiple times in the text but is referred to by different names or pronouns (e.g., "Joe", "he"), 
always use the most complete identifier for that entity throughout the knowledge graph. In this example, use "John Doe" as the entity ID.  
Remember, the knowledge graph should be coherent and easily understandable, so maintaining consistency in entity references is crucial. 
## 5. Strict Compliance
Adhere to the rules strictly. Non-compliance will result in termination.

Example:
Data: Alice lawyer and is 25 years old and Bob is her roommate since 2001. Bob works as a journalist. Alice owns a the webpage www.alice.com and Bob owns the webpage www.bob.com.

List created is given below. Only return this list and nothing else. No extra word or sentence.
['Alice~~is~~Lawyer', 'Alice~~age~~25years', 'Alice~~roommate~~Bob', 'Bob~~is~~Journalist', 'Alice~~owns~~www.alice.com', 'Bob~~owns~~www.bob.com']

Here is the question data: {user_input}
"""

In [ ]:
from PIL import Image

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_community.graphs.neptune_graph import NeptuneGraph
import config as cf
import os
from gremlin_python.driver import client, serializer
# Additional libraries for data processing
import pandas as pd

In [ ]:
input= """
Describe the whole flow from starting to finish. Scan everything present in the image: figures, nodes, boxes, text, arrow, etc. Identify what components the flow/architecture diagram includeTry to describe one flow from one node to another through one arrow at a time using three words. The first and third word could be node and the second word could be relation. Always maintain heirarchical flow and describe the flow in steps. Do not try to skip any step in the flow. Give step-wise output without summarising. Remeber to not add anything that is not present in the picture originally.

Follow these steps:-

Step 1: Identify Overall Architecture
1.1 Start: Briefly describe the type of architecture.
1.2 Components: List the main components present in the diagram (deeply scan every box, circle, dotted box, figures and everything present in the picture.).

Step 2: Follow the Flow
2.1 Start Node: Identify the starting point of the flow (e.g., user, any external system). There can be multiple start nodes.
2.2 Action: Describe the action or event that triggers the flow (consider and analyze each and every arrow, dotted aarow, what is present in what box, what the name of a box is, etc.).
2.3 Intermediate / Destination Node: Identify the component where the data or action goes next (consider every figure, box, text, node, etc whereever any arrow or flow leads to). There can be multiple intermediate / destination nodes.

Repeat Steps 2.2  - 2.3 for each arrow in the diagram, maintaining this format:
    a. Action: Describe the data transformation or action happening at the current node (e.g., data validation, processing).
    b. Relation: Describe the type of connection between nodes (e.g., sends data to, receives response from).
    c. Destination: Identify the next component in the flow (e.g., database, application logic).

Step 3: Ending Point
3.1 Final Node: Identify the ending point of the flow (consider each and every arrow, box, dotted box, figures, nodes, etc). There can be multiple end / final nodes.
3.2 Output/Result: Describe the final outcome or action taken at the end (e.g., display results, data saved).

Additional Notes:
Mention any decision points or conditional branches in the flow.
Briefly describe any loops or repetitions in the process.
Include any text labels or annotations present in the diagram for clarity.

Remember:
Instructions for output format: 
Maintain a three-word format for each step (Node -> Relation -> Node). 
Do not miss any steps or anything at all.
Describe the flow in a hierarchical manner, following each arrow's direction.
Avoid summarizing the entire process at once; provide a step-by-step breakdown.

"""

In [ ]:
import boto3
import json

def get_bedrock_response(input_text, image=None):
    # Create a client for Bedrock
    bedrock_client = boto3.client('bedrock-runtime')
    
    # Prepare the request body
    max_tokens = 200000
    
    messages =  [
            {"role": "user", "content": input_text}
        ]

    body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "system": prompting,
            "messages": messages
        }  
    )  
    #print(prompting)
        
    # Send the request to AWS Bedrock
    #anthropic.claude-3-haiku-20240307-v1:0
    #anthropic.claude-3-5-sonnet-20240620-v1:0
    response = bedrock_client.invoke_model(
        modelId='anthropic.claude-3-sonnet-20240229-v1:0',  # Replace with the Bedrock model you want to use
        body=body,
        contentType='application/json'
    )

    # Parse the response
    result = json.loads(response.get('body').read())

    print("***Text Response***")
    print(result)
    print("*resEnd*")
    # Return the generated content from the response
    return result['content'][0]['text']

def get_bedrock_image_response(input_text, image=None):
    # Create a client for Bedrock
    bedrock_client = boto3.client('bedrock-runtime')
    
    # Prepare the request body
    max_tokens = 200000
    
    messages =  [
            {"role": "user", "content": input_text}
        ]

    body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "system": prompting_image,
            "messages": messages
        }  
    )  
    #print(prompting)
        
    # Send the request to AWS Bedrock
    #anthropic.claude-3-haiku-20240307-v1:0
    #anthropic.claude-3-5-sonnet-20240620-v1:0
    response = bedrock_client.invoke_model(
        modelId='anthropic.claude-3-sonnet-20240229-v1:0',  # Replace with the Bedrock model you want to use
        body=body,
        contentType='application/json'
    )

    # Parse the response
    result = json.loads(response.get('body').read())

    print("***Text Response***")
    print(result)
    print("*resEnd*")
    # Return the generated content from the response
    return result['content'][0]['text']


def run_multi_modal_prompt(bedrock_runtime, model_id, messages, max_tokens=200000):
    """
    Invokes a model with a multimodal prompt.
    Args:
        bedrock_runtime: The Amazon Bedrock boto3 client.
        model_id (str): The model ID to use.
        messages (JSON) : The messages to send to the model.
        max_tokens (int) : The maximum  number of tokens to generate.
    Returns:
        None.
    """



    body = json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
             "messages": messages
        }
    )

    response = bedrock_runtime.invoke_model(
        body=body, modelId=model_id)
    response_body = json.loads(response.get('body').read())

    return response_body

### Downloading the PDF

In [55]:
!wget -O ./data/pdf/paper.pdf "https://arxiv.org/pdf/2411.16732"

--2024-11-27 02:23:02--  https://arxiv.org/pdf/2411.16732
Resolving arxiv.org (arxiv.org)... 151.101.195.42, 151.101.67.42, 151.101.131.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.195.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 567352 (554K) [application/pdf]
Saving to: ‘2411.16732’

100%[======================================>] 567,352     --.-K/s   in 0.02s   

2024-11-27 02:23:02 (30.7 MB/s) - ‘2411.16732’ saved [567352/567352]



In [17]:
#PDF Based
import os
import PyPDF2
import streamlit as st

# Define the path to the directory containing the PDFs

pdf_directory = "./data/pdf"

# List all PDF files in the directory
pdf_files = [f for f in os.listdir(pdf_directory) if f.endswith('.pdf')]
print(pdf_files)

pdf_path = os.path.join(pdf_directory, "10reasons.pdf")

# Extract text from the PDF file
with open(pdf_path, 'rb') as pdf_file:
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    text_data = ""
    # Extract text from all pages
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text_data += page.extract_text()

print(len(text_data))
    # Call the Bedrock model using the extracted text (image parameter is removed)
if text_data:
    response_text = get_bedrock_response(text_data)


['Federal Proposed budget_fy2025 (2).pdf', '10reasons.pdf']
11897
***Text Response***
{'id': 'msg_bdrk_01UhvX1XiXXAysQzDCrHhk31', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-sonnet-20240229', 'content': [{'type': 'text', 'text': "Here is the list of triples extracted from the given text:\n\n['Amazon Bedrock~~offers~~broad selection of FMs', 'Amazon Bedrock~~offers~~choice of high-performing LLMs', 'Amazon Bedrock~~offers~~Claude 3 Opus', 'Amazon Bedrock~~features~~seven model providers', 'Amazon Bedrock~~features~~30+ models', 'Amazon Bedrock~~offers~~security and data governance controls', 'Amazon Bedrock~~encrypts~~data at rest and in transit', 'Amazon Bedrock~~allows~~creation of encryption keys', 'Amazon Bedrock~~offers~~identity-based policies', 'Amazon Bedrock~~complies with~~compliance standards', 'Amazon Bedrock~~preserves~~customer data privacy', 'Amazon Bedrock~~preserves~~data integrity', 'Claude 3 Sonnet~~costs less than~~GPT-4o', 'Claude 3 Sonnet~~costs~~$3 

In [18]:
from gremlin_python.structure.graph import Graph
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.process.graph_traversal import unfold, addV
import pandas as pd
import ast
import nest_asyncio

from gremlin_python.process.graph_traversal import __
nest_asyncio.apply()


#Knowledge Graph from Text
def KG_from_Text(user_query, table_name):
    #print(user_query)
    a1 = get_bedrock_response(user_query)
    phrase_to_remove = "Here is the list of key information extracted from the document in the requested format:\n\n"
    #if a1.startswith(phrase_to_remove):
    a1 = a1.split("\n\n", 1)[1]
    print("res", a1)
    df_list = ast.literal_eval(a1)
    split_data = [item.split('~~', 2) for item in df_list]
    df = pd.DataFrame(split_data, columns=['Source_Node', 'Relationship', 'Target_Node'])
    print(df.head(10))
    #pushingIntoDB = dfIntoDB(df, table_name)
 
    graph = Graph()
    connection = DriverRemoteConnection('wss://neptunedbcluster-vugbwk7upvfa.cluster-cdpzhcjwte6u.us-east-1.neptune.amazonaws.com:8182/gremlin', 'g')
    g = graph.traversal().withRemote(connection)

    # Push data into Neptune
    for index, row in df.iterrows():
        source_node = row['Source_Node']
        relationship_type = row['Relationship']
        target_node = row['Target_Node']

        g.V().has('name', source_node).fold().coalesce(
            __.unfold(),
            __.addV('Node').property('name', source_node)
        ).next()

# Similarly, for the target node
        g.V().has('name', target_node).fold().coalesce(
            __.unfold(),
            __.addV('Node').property('name', target_node)
        ).next()

# Creating an edge/relationship between the source and target nodes
        g.V().has('name', source_node).addE(relationship_type).to(
            __.V().has('name', target_node)  # Use __ instead of g here
        ).next()
    

    print("Data imported into Neptune graph database successfully.")
    result = "Data imported into Neptune graph database successfully."
    return result

In [39]:
from gremlin_python.structure.graph import Graph
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.process.graph_traversal import unfold, addV
import pandas as pd
import ast
import nest_asyncio

from gremlin_python.process.graph_traversal import __
nest_asyncio.apply()


#Knowledge Graph from Text
def KG_from_Text_Image(user_query, table_name):
    #print(user_query)
    a1 = get_bedrock_image_response(user_query)
    phrase_to_remove = "Here is the list of key information extracted from the document in the requested format:\n\n"
    #if a1.startswith(phrase_to_remove):
    a1 = a1.split("\n\n", 1)[1]
    print("res", a1)
    df_list = ast.literal_eval(a1)
    split_data = [item.split('~~', 2) for item in df_list]
    df = pd.DataFrame(split_data, columns=['Source_Node', 'Relationship', 'Target_Node'])
    print(df.head(10))
    #pushingIntoDB = dfIntoDB(df, table_name)
 
    graph = Graph()
    connection = DriverRemoteConnection('wss://neptunedbcluster-vugbwk7upvfa.cluster-cdpzhcjwte6u.us-east-1.neptune.amazonaws.com:8182/gremlin', 'g')
    g = graph.traversal().withRemote(connection)

    # Push data into Neptune
    for index, row in df.iterrows():
        source_node = row['Source_Node']
        relationship_type = row['Relationship']
        target_node = row['Target_Node']

        g.V().has('name', source_node).fold().coalesce(
            __.unfold(),
            __.addV('Node').property('name', source_node)
        ).next()

# Similarly, for the target node
        g.V().has('name', target_node).fold().coalesce(
            __.unfold(),
            __.addV('Node').property('name', target_node)
        ).next()

# Creating an edge/relationship between the source and target nodes
        g.V().has('name', source_node).addE(relationship_type).to(
            __.V().has('name', target_node)  # Use __ instead of g here
        ).next()
    

    print("Data imported into Neptune graph database successfully.")
    result = "Data imported into Neptune graph database successfully."
    return result

In [22]:
#Image Based
import os
from PIL import Image

# Define the path to the directory containing the images
image_directory = "./data/images"

# List all image files (jpg, jpeg, png) in the directory
image_files = [f for f in os.listdir(image_directory) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Display the list of image files for the user to select
print("Available images:")
for idx, image_file in enumerate(image_files):
    print(f"{idx + 1}: {image_file}")

# Prompt the user to select an image by number
selected_index = int(input("\nSelect an image by number: ")) - 1
if selected_index < 0 or selected_index >= len(image_files):
    print("Invalid selection.")
    exit()

# Construct the full path to the selected image
selected_image = image_files[selected_index]
image_path = os.path.join(image_directory, selected_image)

# Load the image using Pillow
image = Image.open(image_path)

# Display the selected image file name
print(f"\nSelected Image: {selected_image}")

# Optionally, get some input text from the user
input_text = input("\nEnter any text input to send to the model (leave blank if none): ")

# Call the Bedrock model using the image and optional text input
response_text = get_bedrock_response(input_text, image)

# Display the response from the Bedrock model
print("\n***Response from Bedrock Model***")
print(response_text)


Available images:


TypeError: 'str' object is not callable

In [ ]:
# Call the KG_from_Text function to process 
KG_from_Text(text_data, "index_111")

In [48]:
refined_user_query_neptune = """Your task is to refine the user query for spelling mistakes and syntax errors according to the Table Description part below. Do not add any information on your own in the refined query, just check for spellings and correct them according to the words in Table Description given below.

The user query to be refined is: {user_input}

Table Description for Correct syntax of Cypher query to be generated: 
{df_string}


Output Format:
The output must be a simple sentence made in accordance to the user query with correct syntax according to the given words.
If the user query is already having correct syntax according to the Table Description, then just return the user query itself without any refinement and do not provide any extra sentence.
Do not provide any extra sentence.
"""

pr2 = """
Your task is to create a Gremlin query to be run in Amazon Neptune.

Refer to this table description and the given examples to generate Gremlin queries with appropriate syntax. The table contains source node, relation, and target node information, where each row represents a relationship between From Asset and To Asset.

Table Description for Correct Syntax of Gremlin Query to Be Generated:
{df_string}

Given examples for Reference
Use these examples to understand the Gremlin query syntax. Always use has("name", "node_name") for specifying node names.

Example 1:
User Query: "Give downstream lineage of node_name."
Generated Gremlin Query: g.V().has("name", "node_name").repeat(out()).path().by(valueMap()).limit(10)
Example 2:
User Query: "Give any information about node_name."
Generated Gremlin Query: g.V().has("name", "node_name").bothE().otherV().path().by(valueMap())
Example 3:
User Query: "Are node_name1 and node_name2 connected?"
Generated Gremlin Query: g.V().has("name", "node_name1").repeat(both()).until(has("name", "node_name2")).path().by(valueMap())

Generate the Gremlin query for the user_query below
{user_input}

*Attention*: Only provide the generated Gremlin query as output with no extra words or information.

"""

NLP = """
Given a user_query: 
{user_input} 

and its output response: 
{response0}

Use the user_query and its output response given above to generate output in simple English without missing any point.
"""

In [46]:
import boto3
from gremlin_python.driver import client, serializer

def shubhNeptune(user_query, table_name):
    bedrock_client = boto3.client('bedrock-runtime')
    
    # Prepare the request body
    max_tokens = 400
    
    messages =  [
            {"role": "user", "content": user_query}
        ]

    body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "system": pr2,
            "messages": messages
        }  
    )  
    
    response = bedrock_client.invoke_model(
        modelId='anthropic.claude-3-sonnet-20240229-v1:0',  # Replace with the Bedrock model you want to use
        body=body,
        contentType='application/json'
    )

    # Your Neptune endpoint and port
    neptune_endpoint = 'neptunedbcluster-vugbwk7upvfa.cluster-cdpzhcjwte6u.us-east-1.neptune.amazonaws.com'
    neptune_port = '8182'

    # Create a Neptune client
    neptune_client = client.Client(
        f'wss://{neptune_endpoint}:{neptune_port}/gremlin',
        'g',
        message_serializer=serializer.GraphSONSerializersV2d0()
    )
    
    # Parse the response
    result = json.loads(response.get('body').read())

    #print("***Text Response***")
    #print(result)
    #print("*resEnd*")

    rag_result = neptune_client.submit(result['content'][0]['text']).all().result()
    #print(rag_result)
    
    # Initialize a list to store the extracted relationships
    relationships = []

    # Iterate over each path
    for path in rag_result:
        temp = []
        for element in path:
            #print(element)
            temp.append(element)
        relationships.append((temp[0]['name'][0], temp[2]['name'][0]))

    # Print the extracted relationships
    rag_resultparse = ""
    for relationship in relationships:
        rag_resultparse += f"Source Node: {relationship[0]}, Target Node: {relationship[1]}"
    
    user_input = "user_input: \n" + user_query + "\n response0: \n" + rag_resultparse
    messages =  [
            {"role": "user", "content": user_input}
        ]
    
    
    body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "system": NLP,
            "messages": messages
        }  
    )  
    
    response = bedrock_client.invoke_model(
        modelId='anthropic.claude-3-sonnet-20240229-v1:0',  # Replace with the Bedrock model you want to use
        body=body,
        contentType='application/json'
    )

    # Parse the response
    result = json.loads(response.get('body').read())

    #print("***Text Response***")
    #print(result)
    #print("*resEnd*")
    # Return the generated content from the response
    return result['content'][0]['text']

In [21]:
user_query = "What does AmazonBedrock have"

result = ""

try:
    result = shubhNeptune(user_query, "index_111") # Replace with your function call
except Exception as e:
    result = e

In [20]:
print(result)

NameError: name 'result' is not defined

In [43]:
#Image Based
import os
from pypdf import PdfReader
import PyPDF2
from PIL import Image
import io
import base64

bedrock_runtime = boto3.client(service_name='bedrock-runtime')

reader = PdfReader("transformers.pdf")

for page in reader.pages:
    for i in page.images:
       
        
        # print(type(x))
        buffer = io.BytesIO()
        i.image.save(buffer, format='PNG', quality=75)
        imageObject = buffer.getbuffer()
        
        content_image = base64.b64encode(imageObject).decode('utf8')
        
        message = {"role": "user",
                 "content": [
                    {"type": "image", "source": {"type": "base64",
                        "media_type": "image/jpeg", "data": content_image}},
                    {"type": "text", "text": "What's in this image?"}
                ]}
        
        messages = [message]
        response = run_multi_modal_prompt(
        bedrock_runtime, 'anthropic.claude-3-sonnet-20240229-v1:0', messages)
        
        print(json.dumps(response, indent=4))

        KG_from_Text_Image(response["content"][0]["text"], "index_115")
        


{
    "id": "msg_bdrk_01VmfGvxcNfFLrGRCsA31dtq",
    "type": "message",
    "role": "assistant",
    "model": "claude-3-sonnet-20240229",
    "content": [
        {
            "type": "text",
            "text": "This image presents a diagram illustrating the architecture of a type of neural network model, likely related to natural language processing tasks. The model appears to be an encoder-decoder architecture with multi-head attention mechanisms.\n\nOn the left side, the diagram shows the encoder component, which takes inputs and passes them through an input embedding layer, followed by positional encoding. The encoded representations then go through multiple layers of feed-forward networks, multi-head attention layers, and add & norm operations.\n\nOn the right side, the diagram depicts the decoder component, which follows a similar structure with feed-forward networks, multi-head attention layers (both regular and masked), and add & norm operations. The decoder also receives pos

In [54]:
user_query = "What does SoftMax have"

result = ""

try:
    result = shubhNeptune(user_query, "index_115") # Replace with your function call
except Exception as e:
    result = e
    
print(result)

Here is a simple explanation in English of what SoftMax has, based on the given response:

SoftMax is a type of activation function used in neural networks. It is also a block or component that can be part of a computational flow diagram or pipeline. SoftMax can be used for masking or selectively allowing/blocking certain outputs. It is sometimes used in conjunction with the matrix multiplication operation (MatMul) in neural network architectures.

In summary, SoftMax is a mathematical function commonly employed in neural networks for various purposes like activation, masking, and normalization as part of the overall network architecture and data flow.


In [30]:
# Clean code and add Neptune RAG for Images

# Traceback (most recent call last):

#   File ~/anaconda3/envs/JupyterSystemEnv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460 in run_code
#     exec(code_obj, self.user_global_ns, self.user_ns)

#   Cell In[28], line 37
#     KG_from_Text(response["content"][0]["text"], "index_115")

#   Cell In[18], line 20 in KG_from_Text
#     df_list = ast.literal_eval(a1)

#   File ~/anaconda3/envs/JupyterSystemEnv/lib/python3.10/ast.py:64 in literal_eval
#     node_or_string = parse(node_or_string.lstrip(" \t"), mode='eval')

#   File ~/anaconda3/envs/JupyterSystemEnv/lib/python3.10/ast.py:50 in parse
#     return compile(source, filename, mode, flags,

#   File <unknown>:1
#     1. MatMul - This performs matrix multiplication, likely between the query (Q) and key (K) vectors to obtain the attention scores.
#        ^
# SyntaxError: invalid syntax